In [1]:
import pandas as pd
import numpy as np
import time
import random
import re
import glob
human_label = pd.read_csv("structured_human_label.csv")
human_label = human_label.dropna()
human_label = human_label.query("target != 'unclear'")
human_label = human_label.query("targeting_type != 'unclear'")
human_label = human_label.query("gender >= 0.75 or race >= 0.75")
human_label = human_label.loc[:, ["comment", "target", "targeting_type", "violence", "derogatory", "profanity", "obscenity", "rejection", "accusation", "gender", "race"]]

In [2]:
def merge_machine_human_df(machine_output):
    machine_output["answer_list"] = machine_output["answer"].str.split('|')
    wider_df = pd.DataFrame()
    for i, row in machine_output.iterrows():
        for attr in row["answer_list"]:
            if attr.split(":")[0].strip() == "target":
                target = attr.split(":")[1].strip()
                row["gpt3_target"] = target
            if attr.split(":")[0].strip() == "targeting type":
                targeting_type = attr.split(":")[1].strip()
                row["gpt3_targeting_type"] = targeting_type
            if attr.split(":")[0].strip() == "violence":
                violence = attr.split(":")[1].strip()
                row["gpt3_violence"] = violence
            if attr.split(":")[0].strip() == "derogatory":
                derogatory = attr.split(":")[1].strip()
                row["gpt3_derogatory"] = derogatory
            if attr.split(":")[0].strip() == "profanity":
                profanity = attr.split(":")[1].strip()
                row["gpt3_profanity"] = profanity
            if attr.split(":")[0].strip() == "obscenities":
                obscenity = attr.split(":")[1].strip()
                row["gpt3_obscenity"] = obscenity
            if attr.split(":")[0].strip() == "rejection":
                rejection = attr.split(":")[1].strip()
                row["gpt3_rejection"] = rejection
            if attr.split(":")[0].strip() == "accusation":
                accusation = attr.split(":")[1].strip()
                row["gpt3_accusation"] = accusation
            if attr.split(":")[0].strip() == "insult":
                insult = attr.split(":")[1].strip()
                row["gpt3_insult"] = insult
        wider_df = wider_df.append(row)
    wider_df = wider_df.dropna()
    df = pd.merge(wider_df, human_label, on = "comment", how = "inner")
    if len(df.columns) < 21:
        df = df[['comment', 'target', 'gpt3_target', 'targeting_type', 'gpt3_targeting_type', 'violence', 'gpt3_violence', 'derogatory', 'profanity', 'obscenity', 'gpt3_insult', 'rejection', 'gpt3_rejection', 'accusation', 'gpt3_accusation', 'gender', 'race']]
    else:
        df = df[['comment', 'target', 'gpt3_target', 'targeting_type', 'gpt3_targeting_type', 'violence', 'gpt3_violence', 'derogatory', 'gpt3_derogatory', 'profanity', 'gpt3_profanity','obscenity', 'gpt3_obscenity', 'rejection', 'gpt3_rejection', 'accusation', 'gpt3_accusation', 'gender', 'race']]
    return df

In [3]:
for filepath in glob.glob('structured_raw/*.csv'):
    filename = filepath.split("/")[1]
    machine_output = pd.read_csv(filepath)
    df = merge_machine_human_df(machine_output)
    print(filename, len(df.columns))
    df.to_csv('structured_merged/'+filename, index = False)
    

structured_results_temp100_attr8.csv 19
structured_results_temp50_attr8.csv 19
structured_results_temp25_attr6.csv 17
structured_results_temp0_attr6.csv 17
structured_results_temp75_attr6.csv 17
structured_results_temp75_attr8.csv 19
structured_results_temp0_attr8.csv 19
structured_results_temp25_attr8.csv 19
structured_results_temp100_attr6.csv 17
structured_results_temp50_attr6.csv 17


In [4]:
def match_answers(df):
    df = df.fillna("NA")
    new_df = pd.DataFrame()
    for i, row in df.iterrows():
        if not row["target"].lower() in row["gpt3_target"].lower() and not row["gpt3_target"].lower() in row["target"].lower():
            #print(row["target"], "|",  row["gpt3_target"],"\n",row["comment"], "\n")
            row["matched_target"] = 0
        else:
            row["matched_target"] = 1

        if not row["targeting_type"].lower() in row["gpt3_targeting_type"].lower() and not row["gpt3_targeting_type"].lower() in row["targeting_type"].lower():
            #print(row["targeting_type"], "|",  row["gpt3_targeting_type"],"\n",row["comment"], "\n")
            row["matched_targeting_type"] = 0
        else:
            row["matched_targeting_type"] = 1

        if (row["violence"] == 1 and row["gpt3_violence"].strip().lower() == "yes") or (row["violence"] == 0 and row["gpt3_violence"].strip().lower() == "no"):
            row["matched_violence"] = 1
        else:
            row["matched_violence"] = 0
            # print(row["violence"], "|",  row["gpt3_violence"],"\n",row["comment"], "\n")

        if (row["rejection"] == 1 and row["gpt3_rejection"].strip().lower() == "yes") or (row["rejection"] == 0 and row["gpt3_rejection"].strip().lower() == "no"):
            row["matched_rejection"] = 1
        else:
            row["matched_rejection"] = 0
            #print(row["rejection"], "|",  row["gpt3_rejection"],"\n",row["comment"], "\n")

        if (row["accusation"] == 1 and row["gpt3_accusation"].strip().lower() == "yes") or (row["accusation"] == 0 and row["gpt3_accusation"].strip().lower() == "no"):
            row["matched_accusation"] = 1
        else:
            row["matched_accusation"] = 0
            #print(row["accusation"], "|",  row["gpt3_accusation"],"\n",row["comment"], "\n")
        
        if len(df.columns) < 19:
            if ((row["derogatory"] == 1 or row["profanity"] == 1 or row["obscenity"] == 1) and row["gpt3_insult"].strip().lower() == "yes") or ((row["derogatory"] == 0 and row["profanity"] == 0 and row["obscenity"] == 0) and row["gpt3_insult"].strip().lower() == "no"):
                row["matched_insult"] = 1
            else:
                row["matched_insult"] = 0
                print(row["derogatory"],row["profanity"],row["obscenity"], "|",  row["gpt3_insult"],"\n",row["comment"], "\n")
        else:
            if (row["derogatory"] == 1 and row["gpt3_derogatory"].strip().lower() == "yes") or (row["derogatory"] == 0 and row["gpt3_derogatory"].strip().lower() == "no"):
                row["matched_derogatory"] = 1
            else:
                row["matched_derogatory"] = 0
                #print(row["derogatory"], "|",  row["gpt3_derogatory"],"\n",row["comment"], "\n")

            if (row["profanity"] == 1 and row["gpt3_profanity"].strip().lower() == "yes") or (row["profanity"] == 0 and row["gpt3_profanity"].strip().lower() == "no"):
                row["matched_profanity"] = 1
            else:
                row["matched_profanity"] = 0
                # print(row["profanity"], "|",  row["gpt3_profanity"],"\n",row["comment"], "\n")

            if (row["obscenity"] == 1 and row["gpt3_obscenity"].strip().lower() == "yes") or (row["obscenity"] == 0 and row["gpt3_obscenity"].strip().lower() == "no"):
                row["matched_obscenity"] = 1
            else:
                row["matched_obscenity"] = 0
                #print(row["obscenity"], "|",  row["gpt3_obscenity"],"\n",row["comment"], "\n")
        new_df = new_df.append(row)
    print(new_df.columns)
    if len(new_df.columns) <= 23:
        new_df = new_df[['comment', 'target', 'gpt3_target', "matched_target", 'targeting_type', 'gpt3_targeting_type', 'matched_targeting_type', 'violence', 'gpt3_violence', 'matched_violence', 'derogatory', 'profanity', 'obscenity', 'gpt3_insult', 'matched_insult', 'rejection', 'gpt3_rejection', 'matched_rejection', 'accusation', 'gpt3_accusation', 'matched_accusation', 'gender', 'race']]
    else:
        new_df = new_df[['comment', 'target', 'gpt3_target', "matched_target", 'targeting_type', 'gpt3_targeting_type', 'matched_targeting_type', 'violence', 'gpt3_violence', 'matched_violence', 'derogatory', 'gpt3_derogatory', 'matched_derogatory', 'profanity', 'gpt3_profanity', 'matched_profanity','obscenity', 'gpt3_obscenity', 'matched_obscenity', 'rejection', 'gpt3_rejection', 'matched_rejection', 'accusation', 'gpt3_accusation', 'matched_accusation', 'gender', 'race']]
        
    return new_df

for filepath in glob.glob('structured_merged/*.csv'):
    filename = filepath.split("/")[1]
    print(filename)
    df = pd.read_csv(filepath)
    new_df = match_answers(df)
    new_df.to_csv(filename, index = False)

structured_results_temp100_attr8.csv
Index(['accusation', 'comment', 'derogatory', 'gender', 'gpt3_accusation',
       'gpt3_derogatory', 'gpt3_obscenity', 'gpt3_profanity', 'gpt3_rejection',
       'gpt3_target', 'gpt3_targeting_type', 'gpt3_violence',
       'matched_accusation', 'matched_derogatory', 'matched_obscenity',
       'matched_profanity', 'matched_rejection', 'matched_target',
       'matched_targeting_type', 'matched_violence', 'obscenity', 'profanity',
       'race', 'rejection', 'target', 'targeting_type', 'violence'],
      dtype='object')
structured_results_temp50_attr8.csv
Index(['accusation', 'comment', 'derogatory', 'gender', 'gpt3_accusation',
       'gpt3_derogatory', 'gpt3_obscenity', 'gpt3_profanity', 'gpt3_rejection',
       'gpt3_target', 'gpt3_targeting_type', 'gpt3_violence',
       'matched_accusation', 'matched_derogatory', 'matched_obscenity',
       'matched_profanity', 'matched_rejection', 'matched_target',
       'matched_targeting_type', 'matched_vio

1.0 0.0 1.0 | no 
 Women!!! Grow a pair and refuse to compete with men!!! This is asinine! 

1.0 1.0 0.0 | no 
 smack the fat ass out of her 

1.0 1.0 0.0 | no 
 Is this world filled by those stupid female bitches? 

1.0 1.0 0.0 | no 
 Feminist deserve to be miserable. What kind of man would want to live with a feminist? Silly me , of course,  a cuck. 

1.0 1.0 1.0 | no 
 Don't hit Women. Just fuck them and make them pregnant. 

1.0 0.0 0.0 | no 
 I wouldn’t like this girl as a police officer. I want police to keep me safe, she and every other woman cannot keep us safe they are week 

0.0 0.0 0.0 | yes 
 I dare you to choke her next time. Every one of them 

1.0 0.0 0.0 | no 
 I hate being a man, wish I was a lady and no man on earth existed 

Index(['accusation', 'comment', 'derogatory', 'gender', 'gpt3_accusation',
       'gpt3_insult', 'gpt3_rejection', 'gpt3_target', 'gpt3_targeting_type',
       'gpt3_violence', 'matched_accusation', 'matched_insult',
       'matched_rejection', '